In [ ]:
# import pandas as pd
# import requests
# from io import StringIO

# # GitHub CSV 파일의 URL
# household_url = ('https://raw.githubusercontent.com/DartB-2024-1st-Toy-Project/Photobooth/main/data/%EC%84%9C%EC%9A%B8%EC%8B%9C%20%EC%83%81%EA%B6%8C%EB%B6%84%EC%84%9D%EC%84%9C%EB%B9%84%EC%8A%A4(%EC%86%8C%EB%93%9D%EC%86%8C%EB%B9%84-%EC%83%81%EA%B6%8C%EB%B0%B0%ED%9B%84%EC%A7%80).csv')

# # Get the content from the URL
# response = requests.get(household_url)

# # Decode the content using different encodings
# data = response.content.decode('utf-8')
# except UnicodeDecodeError:
#     data = response.content.decode('ISO-8859-1')

# # Read the CSV content into a DataFrame
# df = pd.read_csv(StringIO(data))

# # Display the DataFrame
# print(df)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from scipy.stats import norm

In [3]:
# 한글 폰트 설정
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths = None, fontext = 'ttf')
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family = font)

In [8]:
household_df = pd.read_csv('가구특성정보.csv')
university_df = pd.read_csv('대학_중복제거_주소변환.csv')
store_df = pd.read_csv('상가상권정보.csv')
income_df = pd.read_csv('소득소비상권배후지상권분석.csv')
sale_df = pd.read_csv('자치구별추정매출상권분석.csv')
school_df = pd.read_csv('초중고등학교위치.csv')

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\215377036.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  store_df = pd.read_csv('상가상권정보.csv')


#1. Household Dataframe

In [148]:
household_df = pd.read_csv('가구특성정보.csv')

In [149]:
household_df.head(5)

,adstrd_nm,adstrd_cd,legaldong_nm,legaldong_cd,tot_po,tot_hshld_co,hshld_per_po,ave_income_amt
0,서울특별시 종로구 청운효자동,1111051500,청운동,1111010100,11680,4715,2.48,9049.02
1,서울특별시 종로구 청운효자동,1111051500,신교동,1111010200,11680,4715,2.48,5544.59
2,서울특별시 종로구 청운효자동,1111051500,궁정동,1111010300,11680,4715,2.48,7078.69
3,서울특별시 종로구 청운효자동,1111051500,효자동,1111010400,11680,4715,2.48,5403.79
4,서울특별시 종로구 청운효자동,1111051500,창성동,1111010500,11680,4715,2.48,5345.71


In [150]:
household_df.columns
#adstrd_nm == 행정동명
#adstrd_cd == 행정동코드
#legaldong_nm == 법정동명
#legaldong_cd == 법정동코드
#tot_po == 총인구수
#tot hshld co == 총가구수
#hshld_per_po == 가구당 인구 수
#ave income amt == 평균소득금액

Index(['adstrd_nm', 'adstrd_cd', 'legaldong_nm', 'legaldong_cd', 'tot_po',
       'tot_hshld_co', 'hshld_per_po', 'ave_income_amt'],
      dtype='object')

In [151]:
household_df.isnull().sum()

adstrd_nm         0
adstrd_cd         0
legaldong_nm      0
legaldong_cd      0
tot_po            0
tot_hshld_co      0
hshld_per_po      0
ave_income_amt    0
dtype: int64

In [152]:
household_df.nunique()

adstrd_nm          3459
adstrd_cd          3459
legaldong_nm       9645
legaldong_cd      18531
tot_po             3226
tot_hshld_co       2978
hshld_per_po        164
ave_income_amt    17591
dtype: int64

In [153]:
seoul_hhd = household_df[household_df['adstrd_nm'].str.startswith('서울')]

In [155]:
print(seoul_hhd['legaldong_nm'].nunique())
print(seoul_hhd['adstrd_nm'].nunique())
#다른 테이블에 법정동 X. 법정동, 행정동 중에서 행정동 기준으로 함

456
423


In [156]:
# '주소' 컬럼을 공백을 기준으로 분리
seoul_hhd[['시', '구', '동']] = seoul_hhd['adstrd_nm'].str.split(' ', 3).tolist()

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\2770574650.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  seoul_hhd[['시', '구', '동']] = seoul_hhd['adstrd_nm'].str.split(' ', 3).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\2770574650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_hhd[['시', '구', '동']] = seoul_hhd['adstrd_nm'].str.split(' ', 3).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\2770574650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [157]:
seoul_hhd

,adstrd_nm,adstrd_cd,legaldong_nm,legaldong_cd,tot_po,tot_hshld_co,hshld_per_po,ave_income_amt,시,구,동
0,서울특별시 종로구 청운효자동,1111051500,청운동,1111010100,11680,4715,2.48,9049.02,서울특별시,종로구,청운효자동
1,서울특별시 종로구 청운효자동,1111051500,신교동,1111010200,11680,4715,2.48,5544.59,서울특별시,종로구,청운효자동
2,서울특별시 종로구 청운효자동,1111051500,궁정동,1111010300,11680,4715,2.48,7078.69,서울특별시,종로구,청운효자동
3,서울특별시 종로구 청운효자동,1111051500,효자동,1111010400,11680,4715,2.48,5403.79,서울특별시,종로구,청운효자동
4,서울특별시 종로구 청운효자동,1111051500,창성동,1111010500,11680,4715,2.48,5345.71,서울특별시,종로구,청운효자동
...,...,...,...,...,...,...,...,...,...,...,...
725,서울특별시 강동구 성내2동,1174065000,성내동,1174010800,24026,11954,2.01,6010.61,서울특별시,강동구,성내2동
726,서울특별시 강동구 성내3동,1174066000,성내동,1174010800,22682,9716,2.33,6010.61,서울특별시,강동구,성내3동
727,서울특별시 강동구 길동,1174068500,길동,1174010500,44791,19714,2.27,6449.34,서울특별시,강동구,길동
728,서울특별시 강동구 둔촌1동,1174069000,둔촌동,1174010600,36,14,2.57,5761.60,서울특별시,강동구,둔촌1동


In [158]:
seoul_hhd['구_동숫자'] = seoul_hhd['구'] + ' ' + seoul_hhd['동']

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4195091415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_hhd['구_동숫자'] = seoul_hhd['구'] + ' ' + seoul_hhd['동']


In [159]:
print(seoul_hhd['구_동숫자'].nunique())
print(seoul_hhd['동'].nunique())

423
422


In [160]:
# '동' 값이 같은 그룹 선택
same_dong = seoul_hhd.groupby('동')

# '구' 값이 다른 행 찾기
for dong, group in same_dong:
    if len(group['구'].unique()) > 1:
        print(f"동: {dong}")
        print(group)

동: 신사동
         adstrd_nm   adstrd_cd legaldong_nm  legaldong_cd  tot_po  \
621  서울특별시 관악구 신사동  1162068500          신림동    1162010200   26454   
652  서울특별시 강남구 신사동  1168051000          신사동    1168010700   15529   
653  서울특별시 강남구 신사동  1168051000         압구정동    1168011000   15529   

     tot_hshld_co  hshld_per_po  ave_income_amt      시    구    동    구_동숫자  
621         14549          1.82         5017.06  서울특별시  관악구  신사동  관악구 신사동  
652          6621          2.35        12828.44  서울특별시  강남구  신사동  강남구 신사동  
653          6621          2.35        16701.43  서울특별시  강남구  신사동  강남구 신사동  


In [161]:
seoul_hhd.rename(columns={'adstrd_nm': '행정동명', 'adstrd_cd': '행정동코드', 'legaldong_nm': '법정동명', 'legaldong_cd': '법정동코드', 'tot_po': '총인구수', 'tot_hshld_co': '총가구수', 'hshld_per_po': '가구당인구수', 'ave_income_amt': '평균소득금액'}, inplace=True)
seoul_hhd

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\452688903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_hhd.rename(columns={'adstrd_nm': '행정동명', 'adstrd_cd': '행정동코드', 'legaldong_nm': '법정동명', 'legaldong_cd': '법정동코드', 'tot_po': '총인구수', 'tot_hshld_co': '총가구수', 'hshld_per_po': '가구당인구수', 'ave_income_amt': '평균소득금액'}, inplace=True)


,행정동명,행정동코드,법정동명,법정동코드,총인구수,총가구수,가구당인구수,평균소득금액,시,구,동,구_동숫자
0,서울특별시 종로구 청운효자동,1111051500,청운동,1111010100,11680,4715,2.48,9049.02,서울특별시,종로구,청운효자동,종로구 청운효자동
1,서울특별시 종로구 청운효자동,1111051500,신교동,1111010200,11680,4715,2.48,5544.59,서울특별시,종로구,청운효자동,종로구 청운효자동
2,서울특별시 종로구 청운효자동,1111051500,궁정동,1111010300,11680,4715,2.48,7078.69,서울특별시,종로구,청운효자동,종로구 청운효자동
3,서울특별시 종로구 청운효자동,1111051500,효자동,1111010400,11680,4715,2.48,5403.79,서울특별시,종로구,청운효자동,종로구 청운효자동
4,서울특별시 종로구 청운효자동,1111051500,창성동,1111010500,11680,4715,2.48,5345.71,서울특별시,종로구,청운효자동,종로구 청운효자동
...,...,...,...,...,...,...,...,...,...,...,...,...
725,서울특별시 강동구 성내2동,1174065000,성내동,1174010800,24026,11954,2.01,6010.61,서울특별시,강동구,성내2동,강동구 성내2동
726,서울특별시 강동구 성내3동,1174066000,성내동,1174010800,22682,9716,2.33,6010.61,서울특별시,강동구,성내3동,강동구 성내3동
727,서울특별시 강동구 길동,1174068500,길동,1174010500,44791,19714,2.27,6449.34,서울특별시,강동구,길동,강동구 길동
728,서울특별시 강동구 둔촌1동,1174069000,둔촌동,1174010600,36,14,2.57,5761.60,서울특별시,강동구,둔촌1동,강동구 둔촌1동


In [163]:
seoul_hhd

,행정동명,행정동코드,법정동명,법정동코드,총인구수,총가구수,가구당인구수,평균소득금액,시,구,동,구_동숫자
0,서울특별시 종로구 청운효자동,1111051500,청운동,1111010100,11680,4715,2.48,9049.02,서울특별시,종로구,청운효자동,종로구 청운효자동
1,서울특별시 종로구 청운효자동,1111051500,신교동,1111010200,11680,4715,2.48,5544.59,서울특별시,종로구,청운효자동,종로구 청운효자동
2,서울특별시 종로구 청운효자동,1111051500,궁정동,1111010300,11680,4715,2.48,7078.69,서울특별시,종로구,청운효자동,종로구 청운효자동
3,서울특별시 종로구 청운효자동,1111051500,효자동,1111010400,11680,4715,2.48,5403.79,서울특별시,종로구,청운효자동,종로구 청운효자동
4,서울특별시 종로구 청운효자동,1111051500,창성동,1111010500,11680,4715,2.48,5345.71,서울특별시,종로구,청운효자동,종로구 청운효자동
...,...,...,...,...,...,...,...,...,...,...,...,...
725,서울특별시 강동구 성내2동,1174065000,성내동,1174010800,24026,11954,2.01,6010.61,서울특별시,강동구,성내2동,강동구 성내2동
726,서울특별시 강동구 성내3동,1174066000,성내동,1174010800,22682,9716,2.33,6010.61,서울특별시,강동구,성내3동,강동구 성내3동
727,서울특별시 강동구 길동,1174068500,길동,1174010500,44791,19714,2.27,6449.34,서울특별시,강동구,길동,강동구 길동
728,서울특별시 강동구 둔촌1동,1174069000,둔촌동,1174010600,36,14,2.57,5761.60,서울특별시,강동구,둔촌1동,강동구 둔촌1동


In [164]:
import re

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# 데이터프레임의 구_동 칼럼에 적용하여 숫자를 제거한 새로운 칼럼 생성
seoul_hhd['구_동'] = [remove_numbers(address) for address in seoul_hhd['구_동숫자']]

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\2581506764.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_hhd['구_동'] = [remove_numbers(address) for address in seoul_hhd['구_동숫자']]


In [166]:
seoul_hhd['구_동'].nunique()

250

#2. 소득소비상권분석 (서울)

In [65]:
sale_df = pd.read_csv('자치구별추정매출상권분석.csv')

In [68]:
sale_df.head(5)

,기준_년분기_코드,자치구_코드,자치구_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,년,월
0,20191,11110,종로구,CS300028,화초,1495850124,36765,1314004524,181845600,171773552,...,14947,17876,345,10672,9937,6291,4030,1542,2019,1
1,20191,11110,종로구,CS100004,양식음식점,21733503173,586239,15141392928,6592110245,2473306312,...,244199,295978,6781,212438,144750,90539,62719,22943,2019,1
2,20191,11110,종로구,CS100007,치킨전문점,3247811505,111086,2414124197,833687308,387818146,...,60952,43654,2077,34735,25143,20359,16465,5826,2019,1
3,20191,11110,종로구,CS300017,시계및귀금속,150561529485,400097,113674021254,36887508231,23030460391,...,155079,220607,5746,110220,93976,66296,59195,40254,2019,1
4,20191,11110,종로구,CS200006,일반의원,14993800118,538455,13153901584,1839898534,2847348305,...,267337,270382,3352,72058,122236,124040,115864,100169,2019,1


In [61]:
sale_df.columns
#구별 #월별 매출 #요일별 매출 #시간대별 #성별 #연령별

Index(['기준_년분기_코드', '자치구_코드', '자치구_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',
       '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_건수~06_매출_건수',
       '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수',
       '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수',
       '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
       '연령대_50_매출_건수', '연령대_60_이상_매출_건수'],
      dtype='object')

In [62]:
sale_df.isnull().sum()

기준_년분기_코드          0
자치구_코드             0
자치구_코드_명           0
서비스_업종_코드          0
서비스_업종_코드_명        0
당월_매출_금액           0
당월_매출_건수           0
주중_매출_금액           0
주말_매출_금액           0
월요일_매출_금액          0
화요일_매출_금액          0
수요일_매출_금액          0
목요일_매출_금액          0
금요일_매출_금액          0
토요일_매출_금액          0
일요일_매출_금액          0
시간대_00~06_매출_금액    0
시간대_06~11_매출_금액    0
시간대_11~14_매출_금액    0
시간대_14~17_매출_금액    0
시간대_17~21_매출_금액    0
시간대_21~24_매출_금액    0
남성_매출_금액           0
여성_매출_금액           0
연령대_10_매출_금액       0
연령대_20_매출_금액       0
연령대_30_매출_금액       0
연령대_40_매출_금액       0
연령대_50_매출_금액       0
연령대_60_이상_매출_금액    0
주중_매출_건수           0
주말_매출_건수           0
월요일_매출_건수          0
화요일_매출_건수          0
수요일_매출_건수          0
목요일_매출_건수          0
금요일_매출_건수          0
토요일_매출_건수          0
일요일_매출_건수          0
시간대_건수~06_매출_건수    0
시간대_건수~11_매출_건수    0
시간대_건수~14_매출_건수    0
시간대_건수~17_매출_건수    0
시간대_건수~21_매출_건수    0
시간대_건수~24_매출_건수    0
남성_매출_건수           0
여성_매출_건수           0
연령대_10_매출_건수 

In [63]:
sale_df['기준_년분기_코드'].unique().tolist()

[20191,
 20203,
 20221,
 20211,
 20192,
 20193,
 20194,
 20201,
 20202,
 20204,
 20212,
 20213,
 20224,
 20214,
 20222,
 20223,
 20231,
 20233,
 20232]

In [66]:
sale_df['년'] = sale_df['기준_년분기_코드'] // 10
sale_df['월'] = sale_df['기준_년분기_코드'] % 10

,기준_년분기_코드,자치구_코드,자치구_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,년,월
0,20191,11110,종로구,CS300028,화초,1495850124,36765,1314004524,181845600,171773552,...,14947,17876,345,10672,9937,6291,4030,1542,2019,1
1,20191,11110,종로구,CS100004,양식음식점,21733503173,586239,15141392928,6592110245,2473306312,...,244199,295978,6781,212438,144750,90539,62719,22943,2019,1
2,20191,11110,종로구,CS100007,치킨전문점,3247811505,111086,2414124197,833687308,387818146,...,60952,43654,2077,34735,25143,20359,16465,5826,2019,1
3,20191,11110,종로구,CS300017,시계및귀금속,150561529485,400097,113674021254,36887508231,23030460391,...,155079,220607,5746,110220,93976,66296,59195,40254,2019,1
4,20191,11110,종로구,CS200006,일반의원,14993800118,538455,13153901584,1839898534,2847348305,...,267337,270382,3352,72058,122236,124040,115864,100169,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29299,20233,11740,강동구,CS200008,한의원,14615618999,379386,11570948163,3044670836,2761760831,...,166363,211714,419,15484,41340,64151,81700,174983,2023,3
29300,20233,11740,강동구,CS200036,고시원,446829659,5993,228631128,218198531,19340091,...,4036,1957,37,259,2326,1119,1340,912,2023,3
29301,20233,11740,강동구,CS300006,미곡판매,3001108155,108710,2394767318,606340837,467719368,...,42290,64771,0,2067,9576,11553,26819,57047,2023,3
29302,20233,11740,강동구,CS300031,가구,303294683,1092,161063927,142230756,9170250,...,506,466,0,98,70,209,185,414,2023,3


In [35]:
sale_df['서비스_업종_코드_명'].unique()
#사진업 없음. 타 데이터 EDA 후 관련 업종만 뽑아내는 걸로

array(['화초', '양식음식점', '치킨전문점', '시계및귀금속', '일반의원', '의약품', '의료기기', '화장품',
       '철물점', '가방', '고시원', '가전제품수리', '반찬가게', '육류판매', '가구', '패스트푸드점',
       '외국어학원', '서적', '한의원', '일식음식점', '스포츠 강습', '일반의류', '피부관리실', '수산물판매',
       '청과상', '분식전문점', '여관', '섬유제품', '중식음식점', '안경', '컴퓨터및주변장치판매', '골프연습장',
       '완구', '한식음식점', '핸드폰', '신발', '미용실', '자동차수리', '문구', '치과의원', '세탁소',
       '미곡판매', '자동차미용', '일반교습학원', '운동/경기용품', '가전제품', '호프-간이주점', '네일숍',
       '커피-음료', '슈퍼마켓', '자전거 및 기타운송장비', '조명용품', 'PC방', '애완동물', '인테리어',
       '스포츠클럽', '전자상거래업', '당구장', '편의점', '제과점', '노래방', '부동산중개업', '예술학원'],
      dtype=object)

#3. 초중고등학교 위치

In [69]:
school_df.head(5)

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,위도,경도,데이터기준일자
0,B000008068,의귀초등학교,초등학교,1959-02-20,공립,본교,운영,제주특별자치도 서귀포시 남원읍 의귀리 1483-3,제주특별자치도 서귀포시 남원읍 한신로 213,9290000,제주특별자치도교육청,9299000,제주특별자치도서귀포시교육지원청,2013-11-29,2023-07-03,33.311119,126.714628,2023-09-22
1,B000022613,거제장평중학교,중학교,2013-05-01,공립,본교,운영,경상남도 거제시 장평동 148,경상남도 거제시 장평4로 40,9010000,경상남도교육청,9111000,경상남도거제교육지원청,2013-11-29,2023-07-03,34.886086,128.606467,2023-09-22
2,B000022983,연무여자중학교,중학교,1970-04-10,사립,본교,운영,충청남도 논산시 연무읍 동산리 879,충청남도 논산시 연무읍 동안로887번길 5,8140000,충청남도교육청,8222000,충청남도논산계룡교육지원청,2013-11-29,2023-07-03,36.135365,127.104633,2023-09-22
3,B000002721,기장초등학교,초등학교,1911-07-01,공립,본교,운영,부산광역시 기장군 기장읍 동부리 320,부산광역시 기장군 기장읍 읍내로 69,7150000,부산광역시교육청,7211000,부산광역시해운대교육지원청,2013-11-29,2023-07-03,35.245772,129.212766,2023-09-22
4,B000012154,성일여자고등학교,고등학교,1986-05-01,사립,본교,운영,부산광역시 사하구 신평동 60-1,부산광역시 사하구 다대로 161,7150000,부산광역시교육청,7171000,부산광역시서부교육지원청,2013-11-29,2023-07-03,35.089500,128.980063,2023-09-22


In [70]:
school_df.columns

Index(['학교ID', '학교명', '학교급구분', '설립일자', '설립형태', '본교분교구분', '운영상태', '소재지지번주소',
       '소재지도로명주소', '시도교육청코드', '시도교육청명', '교육지원청코드', '교육지원청명', '생성일자', '변경일자',
       '위도', '경도', '데이터기준일자'],
      dtype='object')

In [71]:
school_df.isnull().sum()

학교ID        0
학교명         0
학교급구분       0
설립일자        0
설립형태        0
본교분교구분      0
운영상태        0
소재지지번주소     0
소재지도로명주소    0
시도교육청코드     0
시도교육청명      0
교육지원청코드     0
교육지원청명      0
생성일자        0
변경일자        0
위도          0
경도          0
데이터기준일자     0
dtype: int64

In [73]:
school_df.nunique()
#학교명이 같은 곳이 있음.

학교ID        11989
학교명         10423
학교급구분           3
설립일자         5755
설립형태            3
본교분교구분          2
운영상태            1
소재지지번주소     11315
소재지도로명주소    11273
시도교육청코드        17
시도교육청명         17
교육지원청코드       180
교육지원청명        177
생성일자            2
변경일자           17
위도          11360
경도          11341
데이터기준일자         1
dtype: int64

In [77]:
seoul_school = school_df[school_df['소재지지번주소'].str.startswith('서울')]
seoul_school

,학교ID,학교명,학교급구분,설립일자,설립형태,본교분교구분,운영상태,소재지지번주소,소재지도로명주소,시도교육청코드,시도교육청명,교육지원청코드,교육지원청명,생성일자,변경일자,위도,경도,데이터기준일자
7,B000013479,대영고등학교,고등학교,1987-12-15,공립,본교,운영,서울특별시 영등포구 신길동 336-2,서울특별시 영등포구 도림로86길 16,7010000,서울특별시교육청,7041000,서울특별시남부교육지원청,2013-11-29,2023-07-03,37.503661,126.905866,2023-09-22
11,B000002213,서울양동초등학교,초등학교,1972-08-11,공립,본교,운영,서울특별시 양천구 신정동 941,서울특별시 양천구 오목로23길 24,7010000,서울특별시교육청,7081300,서울특별시강서양천교육지원청,2013-11-29,2023-07-03,37.525848,126.851240,2023-09-22
22,B000002310,서울동작초등학교,초등학교,1983-04-06,공립,본교,운영,서울특별시 동작구 사당동 산9-1,서울특별시 동작구 동작대로29길 214,7010000,서울특별시교육청,7132000,서울특별시동작관악교육지원청,2013-11-29,2023-07-03,37.494102,126.976509,2023-09-22
23,B000012048,서울문영여자고등학교,고등학교,1991-11-07,사립,본교,운영,서울특별시 관악구 봉천동 1695,서울특별시 관악구 관악로 85,7010000,서울특별시교육청,7132000,서울특별시동작관악교육지원청,2013-11-29,2023-07-03,37.472905,126.950918,2023-09-22
24,B000002198,서울갈산초등학교,초등학교,1987-11-26,공립,본교,운영,서울특별시 양천구 신정동 330-10,서울특별시 양천구 목동로 31,7010000,서울특별시교육청,7081300,서울특별시강서양천교육지원청,2013-11-29,2023-07-03,37.512372,126.870230,2023-09-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11774,B000025614,서울하늘숲초등학교,초등학교,2019-03-01,공립,본교,운영,서울특별시 구로구 천왕동 산13-1,서울특별시 구로구 오리로 1106,7010000,서울특별시교육청,7041000,서울특별시남부교육지원청,2013-11-29,2023-07-03,37.484159,126.841418,2023-09-22
11804,B000022815,환일고등학교,고등학교,1947-06-24,사립,본교,운영,서울특별시 마포구 아현동 393-1,서울특별시 중구 환일길 47,7010000,서울특별시교육청,7061000,서울특별시중부교육지원청,2013-11-29,2023-07-03,37.553607,126.962802,2023-09-22
11886,B000026183,서울항동초등학교,초등학교,2019-05-01,공립,본교,운영,서울특별시 구로구 항동 160,서울특별시 구로구 연동로 178,7010000,서울특별시교육청,7041000,서울특별시남부교육지원청,2013-11-29,2023-07-03,37.477016,126.824110,2023-09-22
11919,B000002166,서울강덕초등학교,초등학교,2019-09-02,공립,본교,운영,서울특별시 강동구 고덕동 222,서울특별시 강동구 동남로82길 94-54,7010000,서울특별시교육청,7130000,서울특별시강동송파교육지원청,2013-11-29,2023-07-03,37.559547,127.160484,2023-09-22


In [84]:
# '주소' 컬럼을 공백을 기준으로 분리
seoul_school[['시', '구', '동', '이외주소']] = seoul_school['소재지지번주소'].str.split(' ', 3).tolist()

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4068727661.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  seoul_school[['시', '구', '동', '이외주소']] = seoul_school['소재지지번주소'].str.split(' ', 4).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4068727661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_school[['시', '구', '동', '이외주소']] = seoul_school['소재지지번주소'].str.split(' ', 4).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4068727661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [109]:
seoul_school['구_동'] = seoul_school['구'] + ' ' + seoul_school['동']

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\3711604838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_school['구_동'] = seoul_school['구'] + ' ' + seoul_school['동']


In [110]:
print(seoul_school['구_동'].nunique())
print(seoul_school['동'].nunique())
print(seoul_school.nunique())
#다른 구에 겹치는 동 이름이 있음. 동의 경우 구_동으로 묶을 것
#seoul_hhd와 seoul_hh은 똑같은 구_동 개수가 있음.
#서울 내에는 겹치는 학교명이 없음.

243
242
학교ID        1313
학교명         1313
학교급구분          3
설립일자        1033
설립형태           3
본교분교구분         1
운영상태           1
소재지지번주소     1161
소재지도로명주소    1158
시도교육청코드        1
시도교육청명         1
교육지원청코드       14
교육지원청명        11
생성일자           2
변경일자           5
위도          1180
경도          1177
데이터기준일자        1
시              1
구             25
동            242
이외주소        1038
구_동          243
dtype: int64


In [113]:
#데이터기준일자 drop, 운영상태 drop, 생성일자 drop, 변경일자 drop, 본교분교구분 drop
seoul_school.drop(columns=['본교분교구분', '데이터기준일자', '운영상태', '시도교육청코드', '시도교육청명', '생성일자', '변경일자'], inplace=True)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4285655950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_school.drop(columns=['본교분교구분', '데이터기준일자', '운영상태', '시도교육청코드', '시도교육청명', '생성일자', '변경일자'], inplace=True)


In [ ]:
import csv
import requests

# API 정보 셋팅
url = "https://dapi.kakao.com/v2/local/search/address.json"
REST_API_KEY = "2f55e6eb59d65a3dbf95fa448046683d"
headers = {"Authorization": "KakaoAK {}".format(REST_API_KEY)}

def get_jibun_address(road_address):
    # Kakao Maps API 호출
    response = requests.get(url, params={"query": road_address}, headers=headers)
    data = response.json()

    if 'documents' in data and data['documents']:
        jibun_address = data['documents'][0]['address']['address_name']
        return jibun_address
    else:
        return None

def convert_csv_addresses(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        with open(output_file, 'w', encoding='utf-8', newline='') as output_csv:
            csv_writer = csv.DictWriter(output_csv, fieldnames=csv_reader.fieldnames + ['지번주소'])
            csv_writer.writeheader()

            for row in csv_reader:
                road_address = row['소재지도로명주소']

                jibun_address = get_jibun_address(road_address)

                if jibun_address:
                    row['지번주소'] = jibun_address
                else:
                    print(f"지번 주소를 찾을 수 없습니다: {road_address}")

                csv_writer.writerow(row)

# 입력 및 출력 파일명 지정
input_csv_file = "전국대학및전문대학정보표준데이터_중복제거.csv"  # 입력 CSV 파일명
output_csv_file = "전국대학및전문대학정보표준데이터_주소변환.csv"  # 출력 CSV 파일명

# 주소 변환을 수행하여 새로운 CSV 파일 생성
convert_csv_addresses(input_csv_file, output_csv_file)

#소득소비

In [47]:
income_df.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,20232,A,골목상권,3110038,경향신문사,5911238,9,2.062929e+08,4.927222e+07,2.773956e+07,1.656598e+07,2.482017e+07,3.634885e+07,12129783.0,6042386.0,2.475644e+07,8617504.0
1,20232,A,골목상권,3110231,장안교,2729364,6,4.238212e+09,1.091999e+09,5.120422e+08,3.064482e+08,5.376488e+08,6.970880e+08,166550715.0,122336748.0,6.281456e+08,175952776.0
2,20231,A,골목상권,3110020,서울국제고등학교,2977019,6,6.426080e+09,1.602533e+09,8.299889e+08,4.583267e+08,7.960414e+08,9.197401e+08,239423030.0,226293231.0,1.049115e+09,304617898.0
3,20232,A,골목상권,3110824,대방역 3번,4238429,8,5.494606e+09,1.352687e+09,6.656540e+08,3.922016e+08,6.593435e+08,9.221200e+08,204995399.0,158457683.0,9.037307e+08,235415460.0
4,20232,A,골목상권,3110897,서울대입구역 8번,2639874,6,1.299916e+10,3.425036e+09,1.651944e+09,1.050655e+09,1.585843e+09,2.013364e+09,482165601.0,467319303.0,1.714708e+09,608129251.0


In [48]:
income_df.columns
#다른 데이터프레임과 병렬이 어려움. 위도, 경도 따오기 필요!

Index(['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '월_평균_소득_금액',
       '소득_구간_코드', '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액',
       '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액', '교육_지출_총금액',
       '유흥_지출_총금액'],
      dtype='object')

#5. 대학

In [135]:
university_df = pd.read_csv('대학_중복제거_주소변환.csv')
university_df

,학교명,학교 영문명,본분교구분명,대학구분명,학교구분명,설립형태구분명,시도코드,시도명,소재지도로명주소,소재지지번주소,...,소재지우편번호,홈페이지주소,대표전화번호,대표팩스번호,설립일자,기준연도,데이터기준일자,제공기관코드,제공기관명,지번주소
0,한신대학교,Hanshin University,본교,대학,대학교,사립,41,경기도,"경기도 오산시 한신대길 137 (양산동, 한신대학교)",NaN,...,NaN,www.hs.ac.kr,031-379-0114,031-379-0039,1940-04-19,2022,2023-12-27,B340014,한국대학교육협의회,경기 오산시 양산동 400
1,한세대학교,Hansei University,본교,대학,대학교,사립,41,경기도,경기도 군포시 한세로 30 (당정동),NaN,...,NaN,www.hansei.ac.kr,031-450-5114,031-450-5124,1953-05-10,2022,2023-12-27,B340014,한국대학교육협의회,경기 군포시 당정동 964-1
2,성결대학교,Sungkyul University,본교,대학,대학교,사립,41,경기도,"경기도 안양시 만안구 성결대학로 53 (안양동, 성결대학교)",NaN,...,NaN,www.sungkyul.ac.kr,031-467-8114,031-449-0529,1965-01-13,2022,2023-12-27,B340014,한국대학교육협의회,경기 안양시 만안구 안양동 400-10
3,평택대학교,Pyeongtaek University,본교,대학,대학교,사립,41,경기도,"경기도 평택시 서동대로 3825 (용이동, 평택대학교)",NaN,...,NaN,www.ptu.ac.kr,031-659-8114,031-659-8011,1990-11-28,2022,2023-12-27,B340014,한국대학교육협의회,경기 평택시 용이동 605
4,성공회대학교,Sungkonghoe University,본교,대학,대학교,사립,11,서울특별시,"서울특별시 구로구 연동로 320 (항동, 성공회대학교)",NaN,...,NaN,www.skhu.ac.kr,02-2610-4114,02-2610-4248,1992-12-23,2022,2023-12-27,B340014,한국대학교육협의회,서울 구로구 항동 1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,한국전통문화대학교 미래문화유산대학원,The Graduate School of Cultural Heritage for t...,본교,대학원,특수대학원,특별법국,44,충청남도,"세종특별자치시 다정중앙로 20(다정동, 가온마을7단지)",NaN,...,NaN,www.nuch.ac.kr,041-830-7114,041-830-7020,2022-02-28,2022,2023-12-27,B340014,한국대학교육협의회,세종특별자치시 다정동 965
436,한신대학교 사회혁신경영대학원,"The Graduate Of Social Innovation Business, Ha...",본교,대학원,특수대학원,사립,11,서울특별시,"서울특별시 강북구 인수봉로 159 (수유동, 한신대학교신학대학원)",NaN,...,NaN,http://www.hs.ac.kr/hsmba/index.do,02-2125-0114,02-2125-0108,2014-03-06,2022,2023-12-27,B340014,한국대학교육협의회,서울 강북구 수유동 498-16
437,호서대학교 글로벌창업대학원,Hoseo Graduate School Of Enterpreneurship,본교,대학원,특수대학원,사립,11,서울특별시,서울특별시 서초구 남부순환로 2497 (서초동),NaN,...,NaN,gradgate.hoseo.ac.kr,02-2059-2313,02-2055-1405,2004-11-24,2022,2023-12-27,B340014,한국대학교육협의회,서울 서초구 서초동 1422-7
438,호서대학교 벤처전문대학원,Graduate School Of Venture,본교,대학원,전문대학원,사립,11,서울특별시,서울특별시 서초구 반포대로 9 (서초동),NaN,...,NaN,gsv.hoseo.edu,02-2059-2313,02-2055-1405,1999-11-02,2022,2023-12-27,B340014,한국대학교육협의회,서울 서초구 서초동 1463-10


In [136]:
#전체 440 row
university_df.isnull().sum()
university_df.drop(columns=['소재지지번주소', '소재지우편번호'], inplace=True)

In [137]:
university_df.columns

Index(['학교명', '학교 영문명', '본분교구분명', '대학구분명', '학교구분명', '설립형태구분명', '시도코드', '시도명',
       '소재지도로명주소', '도로명우편번호', '홈페이지주소', '대표전화번호', '대표팩스번호', '설립일자', '기준연도',
       '데이터기준일자', '제공기관코드', '제공기관명', '지번주소'],
      dtype='object')

In [138]:
#시도명으로 하면 충남이 섞임;
seoul_univ = university_df[university_df['지번주소'].str.startswith('서울')]

In [139]:
#서울 먼저 추리고 split 해야 함.
# '주소' 컬럼을 공백을 기준으로 분리
seoul_univ[['시', '구', '동', '이외주소']] = seoul_univ['지번주소'].str.split(' ', 3).tolist()

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\1202773657.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  seoul_univ[['시', '구', '동', '이외주소']] = seoul_univ['지번주소'].str.split(' ', 3).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\1202773657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_univ[['시', '구', '동', '이외주소']] = seoul_univ['지번주소'].str.split(' ', 3).tolist()
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\1202773657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [140]:
seoul_univ

,학교명,학교 영문명,본분교구분명,대학구분명,학교구분명,설립형태구분명,시도코드,시도명,소재지도로명주소,도로명우편번호,...,설립일자,기준연도,데이터기준일자,제공기관코드,제공기관명,지번주소,시,구,동,이외주소
4,성공회대학교,Sungkonghoe University,본교,대학,대학교,사립,11,서울특별시,"서울특별시 구로구 연동로 320 (항동, 성공회대학교)",8359,...,1992-12-23,2022,2023-12-27,B340014,한국대학교육협의회,서울 구로구 항동 1-1,서울,구로구,항동,1-1
5,태재대학교,TAEJAE UNIVERSITY,본교,대학,사이버대학(대학),사립,11,서울특별시,서울특별시 종로구 창덕궁5길 22-8 (원서동),3051,...,2023-09-01,2022,2023-12-27,B340014,한국대학교육협의회,서울 종로구 원서동 9-4,서울,종로구,원서동,9-4
12,가톨릭대학교,The Catholic University Of Korea,제2캠퍼,대학,대학교,사립,11,서울특별시,"서울특별시 서초구 반포대로 222 (반포동, 가톨릭대학교성의교정)",6591,...,1995-03-01,2022,2023-12-27,B340014,한국대학교육협의회,서울 서초구 반포동 505,서울,서초구,반포동,505
13,가톨릭대학교,The Catholic University Of Korea,제3캠퍼,대학,대학교,사립,11,서울특별시,"서울특별시 종로구 창경궁로 296-12 (혜화동, 가톨릭대학교성신교정)",3083,...,1995-03-01,2022,2023-12-27,B340014,한국대학교육협의회,서울 종로구 혜화동 90-1,서울,종로구,혜화동,90-1
14,감리교신학대학교,Methodist Theological University,본교,대학,대학교,사립,11,서울특별시,"서울특별시 서대문구 독립문로 56 (냉천동, 감리교신학대학)",3745,...,1887-09-29,2022,2023-12-27,B340014,한국대학교육협의회,서울 서대문구 냉천동 31,서울,서대문구,냉천동,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,백석대학교 교육대학원,"Graduate School Of Education, Baekseok University",본교,대학원,특수대학원,사립,11,서울특별시,"서울특별시 서초구 방배로9길 15 (방배동, 목양동백석대학교)",6695,...,1999-11-19,2022,2023-12-27,B340014,한국대학교육협의회,서울 서초구 방배동 981-61,서울,서초구,방배동,981-61
425,서울대학교 의학대학원,School Of Medicine,본교,대학원,전문대학원,국립대법,11,서울특별시,"서울특별시 종로구 대학로 103 (연건동, 서울대학교 의과대학)",3080,...,2008-12-05,2022,2023-12-27,B340014,한국대학교육협의회,서울 종로구 연건동 28-2,서울,종로구,연건동,28-2
436,한신대학교 사회혁신경영대학원,"The Graduate Of Social Innovation Business, Ha...",본교,대학원,특수대학원,사립,11,서울특별시,"서울특별시 강북구 인수봉로 159 (수유동, 한신대학교신학대학원)",1025,...,2014-03-06,2022,2023-12-27,B340014,한국대학교육협의회,서울 강북구 수유동 498-16,서울,강북구,수유동,498-16
437,호서대학교 글로벌창업대학원,Hoseo Graduate School Of Enterpreneurship,본교,대학원,특수대학원,사립,11,서울특별시,서울특별시 서초구 남부순환로 2497 (서초동),6724,...,2004-11-24,2022,2023-12-27,B340014,한국대학교육협의회,서울 서초구 서초동 1422-7,서울,서초구,서초동,1422-7


In [141]:
seoul_univ['구_동'] = seoul_univ['구'] + ' ' + seoul_univ['동']

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\4261946959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_univ['구_동'] = seoul_univ['구'] + ' ' + seoul_univ['동']


In [142]:
seoul_univ.isnull().sum()

학교명         0
학교 영문명      0
본분교구분명      0
대학구분명       0
학교구분명       0
설립형태구분명     0
시도코드        0
시도명         0
소재지도로명주소    0
도로명우편번호     0
홈페이지주소      1
대표전화번호      0
대표팩스번호      0
설립일자        0
기준연도        0
데이터기준일자     0
제공기관코드      0
제공기관명       0
지번주소        0
시           0
구           0
동           0
이외주소        0
구_동         0
dtype: int64

In [143]:
#학교명 겹치는 곳 있음
seoul_univ.nunique()

학교명         68
학교 영문명      68
본분교구분명       3
대학구분명        3
학교구분명       10
설립형태구분명      5
시도코드         1
시도명          1
소재지도로명주소    69
도로명우편번호     69
홈페이지주소      67
대표전화번호      68
대표팩스번호      68
설립일자        64
기준연도         1
데이터기준일자      1
제공기관코드       1
제공기관명        1
지번주소        68
시            1
구           20
동           59
이외주소        67
구_동         59
dtype: int64

In [145]:
seoul_univ.isnull().sum()
seoul_univ.drop(columns=['홈페이지주소', '데이터기준일자', '제공기관코드', '제공기관명', '기준연도', '시도코드', '시도명', '대표전화번호', '대표팩스번호'], inplace=True)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_22752\2542192481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_univ.drop(columns=['홈페이지주소', '데이터기준일자', '제공기관코드', '제공기관명', '기준연도', '시도코드', '시도명', '대표전화번호', '대표팩스번호'], inplace=True)


In [146]:
seoul_univ
#사이버 대학 있음! 제외 고려
#지번주소 -> 구, 동 가져왔는데 적은 건 대학이 없는 동이 많아서임.

,학교명,학교 영문명,본분교구분명,대학구분명,학교구분명,설립형태구분명,소재지도로명주소,도로명우편번호,설립일자,지번주소,시,구,동,이외주소,구_동
4,성공회대학교,Sungkonghoe University,본교,대학,대학교,사립,"서울특별시 구로구 연동로 320 (항동, 성공회대학교)",8359,1992-12-23,서울 구로구 항동 1-1,서울,구로구,항동,1-1,구로구 항동
5,태재대학교,TAEJAE UNIVERSITY,본교,대학,사이버대학(대학),사립,서울특별시 종로구 창덕궁5길 22-8 (원서동),3051,2023-09-01,서울 종로구 원서동 9-4,서울,종로구,원서동,9-4,종로구 원서동
12,가톨릭대학교,The Catholic University Of Korea,제2캠퍼,대학,대학교,사립,"서울특별시 서초구 반포대로 222 (반포동, 가톨릭대학교성의교정)",6591,1995-03-01,서울 서초구 반포동 505,서울,서초구,반포동,505,서초구 반포동
13,가톨릭대학교,The Catholic University Of Korea,제3캠퍼,대학,대학교,사립,"서울특별시 종로구 창경궁로 296-12 (혜화동, 가톨릭대학교성신교정)",3083,1995-03-01,서울 종로구 혜화동 90-1,서울,종로구,혜화동,90-1,종로구 혜화동
14,감리교신학대학교,Methodist Theological University,본교,대학,대학교,사립,"서울특별시 서대문구 독립문로 56 (냉천동, 감리교신학대학)",3745,1887-09-29,서울 서대문구 냉천동 31,서울,서대문구,냉천동,31,서대문구 냉천동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,백석대학교 교육대학원,"Graduate School Of Education, Baekseok University",본교,대학원,특수대학원,사립,"서울특별시 서초구 방배로9길 15 (방배동, 목양동백석대학교)",6695,1999-11-19,서울 서초구 방배동 981-61,서울,서초구,방배동,981-61,서초구 방배동
425,서울대학교 의학대학원,School Of Medicine,본교,대학원,전문대학원,국립대법,"서울특별시 종로구 대학로 103 (연건동, 서울대학교 의과대학)",3080,2008-12-05,서울 종로구 연건동 28-2,서울,종로구,연건동,28-2,종로구 연건동
436,한신대학교 사회혁신경영대학원,"The Graduate Of Social Innovation Business, Ha...",본교,대학원,특수대학원,사립,"서울특별시 강북구 인수봉로 159 (수유동, 한신대학교신학대학원)",1025,2014-03-06,서울 강북구 수유동 498-16,서울,강북구,수유동,498-16,강북구 수유동
437,호서대학교 글로벌창업대학원,Hoseo Graduate School Of Enterpreneurship,본교,대학원,특수대학원,사립,서울특별시 서초구 남부순환로 2497 (서초동),6724,2004-11-24,서울 서초구 서초동 1422-7,서울,서초구,서초동,1422-7,서초구 서초동
